In [2]:
import numpy as np

#size of X_seen is 40 * (N_i x D) where N_i is the number of training samples for ith seen class
X_seen = np.load('X_seen.npy',encoding='bytes',allow_pickle=True)

#40x85 matrix with each row being the 85-dimensional class attribute vector of a seen class.
class_attributes_seen = np.load('class_attributes_seen.npy',encoding='bytes',allow_pickle=True)

#10x85 matrix with each row being the 85-dimensional class attribute vector of an  unseen class.
class_attributes_unseen = np.load('class_attributes_unseen.npy',encoding='bytes',allow_pickle=True)

#loading test data feature matrices
Xtest=np.load('Xtest.npy',encoding='bytes',allow_pickle=True)

#loading true labels of test data
Ytest=np.load('Ytest.npy',encoding='bytes',allow_pickle=True)


mean_X_seen = np.empty((0,4096))
S_ck = np.empty((0,40))
normalized_S_ck = np.empty((0,40))
predicted_labels = np.empty((0,1))

#calculating feature means of all seen 40 classes
for seen_matrix in X_seen:
    abc = np.array([])
    for i in range(seen_matrix.shape[1]):
        abc = np.append(abc,np.sum(seen_matrix[:,i])/seen_matrix.shape[0])
    mean_X_seen = np.vstack((mean_X_seen,abc))
#print(mean_X_seen)
  
#calculating similarity between each seen class and each unseen class feature vector
for c in range(len(class_attributes_unseen)):
    lst = np.array([])
    for k in range(len(class_attributes_seen)):
        lst = np.append(lst,class_attributes_unseen[c] @ class_attributes_seen[k])
    S_ck = np.vstack((S_ck,lst))
    
#normalizing the vector S_ck
for vector in S_ck:
    lyst = np.array([])
    x = sum(vector)
    for feature_val in vector:
        lyst = np.append(lyst,feature_val/x)
    normalized_S_ck = np.vstack((normalized_S_ck,lyst))
#print(normalized_S_ck)

#calculating mean for unseen 10 classes
mean_X_unseen = normalized_S_ck @ mean_X_seen
#print(mean_X_unseen)


#calculating predicted labels
for test in Xtest:
    efg = np.array([])
    for unseen_mean in mean_X_unseen:
        y = np.linalg.norm(test - unseen_mean)
        efg = np.append(efg,y)
    predicted_labels = np.vstack((predicted_labels,np.argmin(efg)+1))

#testing accuracy
correct_predictions = np.sum(Ytest == predicted_labels)
total_predictions = Ytest.shape[0]
accuracy = correct_predictions / total_predictions
print("Accuracy:",accuracy*100,"%")

Accuracy: 46.89320388349515 %
